In [4]:
pip install selenium

  Using cached selenium-4.19.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached trio-0.25.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached selenium-4.19.0-py3-none-any.whl (10.5 MB)
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ---------------------- ----------------- 92.2/163.8 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 163.8/163.8 kB 3.3 MB/s eta 0:00:00
Using cached trio-0.25.0-py3-none-any.whl (467 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
   ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
   ---------------------------------------- 121.1/121.1 kB 6.9 MB/s et

In [5]:
pip install webdriver-manager

  Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.1-py2.py3-none-any.whl (27 kB)
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

from webdriver_manager.chrome import ChromeDriverManager

def find_element_or_none(doc, selector: str):
    try:
        return doc.find_element(By.CSS_SELECTOR, selector).text
    except (NoSuchElementException, StaleElementReferenceException):
        return None

def scrape_job_offers(url):
    # Setup Chrome options
    chrome_options = Options()
    # Runs Chrome in headless mode.
    # chrome_options.add_argument("--headless")

    # Setup Selenium with ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Navigate to the page
    driver.get(url)
    
    # Selector list of offers
    sel_job_list = '#jsJobResContent .jobItem'
    
    # Selectors fields for each item in the list of offers
    sel_job_item_title = '.jobResultsTitle'
    sel_job_item_salary = '.jobResultsSalary'
    sel_job_item_loc = '.jobResultsLoc'
    sel_job_item_type = '.jobResultsType'
    sel_job_item_when = '.when'

    # Right panel containing all job details: including description, location, industry
    sel_job_detail_container = '.main_detail_content'
    
    # Job details description
    sel_job_detail_skills = '#md_skills'

    # Job details fields after the description
    # sel_job_detail_location = '#md_location'
    # sel_job_detail_industry = '#md_industry'
    # sel_job_detail_category= '#md_category'
    sel_job_detail_duration = '#md_items'
    sel_job_detail_start_date = '#md_start_date'
    #sel_job_detail_rate= '#md_rate'
    sel_job_detail_recruiter= '#md_recruiter'
    sel_job_detail_ref= '#md_ref'
    sel_job_detail_posted_date = '#md_posted_date'
    sel_job_detail_permalink = '#md_permalink'
            
    # Find all job offer items in the list
    job_items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, sel_job_list)))
    print(f"Job_tems found: {len(job_items)}")
    
    for index, job_item in enumerate(job_items):
        # Click on the job item to load its details into the div; may need to scroll into view
        ActionChains(driver).move_to_element(job_item).click(job_item).perform()

        # Extract details from the div
        title = find_element_or_none(job_item, sel_job_item_title)
        salary = find_element_or_none(job_item, sel_job_item_salary)
        loc = find_element_or_none(job_item, sel_job_item_loc)
        type = find_element_or_none(job_item, sel_job_item_type)
        when = find_element_or_none(job_item, sel_job_item_when)

        job_detail_container = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, sel_job_detail_container)))
        print(f"job_detail_container: {job_detail_container.text}")

        #driver.execute_script("arguments[0].scrollIntoView();", sel_job_detail_permalink)

        # field_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, f"//div[@id='{sel_job_detail_permalink}']//*")))
        
        element = driver.find_element(sel_job_detail_permalink)
        ActionChains(driver).move_to_element(element).perform()

        skills = find_element_or_none(job_detail_container, sel_job_detail_skills)
        duration = find_element_or_none(job_detail_container, sel_job_detail_duration)
        start_date = find_element_or_none(job_detail_container, sel_job_detail_start_date)
        #rate = find_element_or_none(job_detail_container, sel_job_detail_rate)
        recruiter = find_element_or_none(job_detail_container, sel_job_detail_recruiter)
        ref = find_element_or_none(job_detail_container, sel_job_detail_ref)
        posted_date = find_element_or_none(job_detail_container, sel_job_detail_posted_date)
        permalink = find_element_or_none(job_detail_container, sel_job_detail_permalink)

        print(f"*Job offer {index + 1}:\n{title}\n{salary}\n{loc}\n{type}\n{when}\n")
        print(f"*Job details: {skills}\nduration:{duration}\nstart_date:{start_date}\nrecruiter:{recruiter}\nref:{ref}\nposted_date:{posted_date}\npermalink:{permalink}\n")
        
        
    # Close the browser
    driver.quit()

if __name__ == "__main__":
    url = "https://www.jobserve.com/gb/en/JobSearch.aspx?shid=2981D9787360CBB0C8BE"
    scrape_job_offers(url)
    print("Ended")


Job_tems found: 25
job_detail_container: See more jobs like this
Applicants must be eligible to work in the specified location
Machine Learning Engineer with AWS - London
We are looking for an experienced hands-on AWS Architect/Cloud Engineer to join a critical project with a Fintech Giant in London. As the Machine Learning expert, you'll be leading teams that are architecting and building complex systems on AWS infrastructure that will be industry changing.


InvalidArgumentException: Message: invalid argument: invalid locator
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00504CC3+225091]
	(No symbol) [0x00434E11]
	(No symbol) [0x002D9A7A]
	(No symbol) [0x00311689]
	(No symbol) [0x0031188B]
	(No symbol) [0x00347882]
	(No symbol) [0x0032F5A4]
	(No symbol) [0x00345CB0]
	(No symbol) [0x0032F2F6]
	(No symbol) [0x003079B9]
	(No symbol) [0x0030879D]
	sqlite3_dbdata_init [0x00979A63+4064547]
	sqlite3_dbdata_init [0x0098106A+4094762]
	sqlite3_dbdata_init [0x0097B968+4072488]
	sqlite3_dbdata_init [0x0067C9C9+930953]
	(No symbol) [0x004407E4]
	(No symbol) [0x0043AD08]
	(No symbol) [0x0043AE31]
	(No symbol) [0x0042CAA0]
	BaseThreadInitThunk [0x761BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77587C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77587C2E+238]
